## Core

- Prepare files for flux measurements:

    - Primary beam correction
    - Convolution to worse resolution
    - Regrid to common frame



In [3]:
import numpy as np
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
import convRegr
from prettytable import PrettyTable


fint = fluxInt.flInt()
cvReg = convRegr.cvRegr()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/Core/'
imDir = rootDir+'inIms/'

fint.rootdir = rootDir
syn.rootdir = rootDir
pb.rootdir = rootDir

aut='KK'
tableName = rootDir+'coreFileNames'+aut+'.tbl'
tableFileNames= ascii.read(tableName,format='csv')

tableNameOut = rootDir+'coreFileNamesCvRg'+aut+'.tbl'

print tableFileNames.colnames

print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


['FileName', 'Frequency', 'Noise[Jy/beam]', 'ErrPerc']
	+---------+
	 Inputs loaded
	+---------+


In [4]:
#-------------------------------------------------#
# Primary beam correction
#-------------------------------------------------#

fileNames=np.empty([len(tableFileNames.columns[0])],dtype='object')

for i in xrange(0,len(tableFileNames.columns[0])):
    fileName = rootDir+tableFileNames.columns[0][i]
    if (fileName == '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_101GHz.fits' or 
       fileName ==  '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_1500MHz.fits' or
       fileName== '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_103GHz.fits' or
       fileName== '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_113GHz.fits'):
        fileNames[i] = fileName
        continue
    ff=fits.open(fileName)
    dats=ff[0].data
    heads=ff[0].header
    dats=np.squeeze(dats)
    
    if i <2:
        telescope= 'MeerKAT'
    elif i ==2 and i==3:
        telescope= 'VLA'
    elif i ==4: 
        telescope = 'ACA'
        
    fileNamePbCorr = pb.BeamCorrect(fileName,dats,heads,telescope)
    fileNames[i] = fileNamePbCorr
    
print fileNames    
print '''\t+---------+\n\t Primary beam correction done\n\t+---------+'''
 

['/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Meer_1030MHz_pbcorr.fits'
 '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Meer_1440MHz_pbcorr.fits'
 '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_4860MHz_pbcorr.fits'
 '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_15GhzB_pbcorr.fits'
 '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_101GHz.fits'
 '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_103GHz.fits'
 '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_113GHz.fits']
	+---------+
	 Primary beam correction done
	+---------+


In [5]:
#-------------------------------------------------#
# Convolve to worse resolution and reproject
#-------------------------------------------------#
fileBaseRg = '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_4860MHz_pbcorr.fits'
fileBaseCv = '/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_101GHz.fits'

a = fits.open(fileBaseCv)
headBeam = a[0].header
mainBeam = np.array([headBeam['BMAJ'],headBeam['BMIN']])
rgFileNames=[]
for i in xrange(0,len(fileNames)):
    
    fileName = fileNames[i]
    #if i == 3:
    #    telescope = 'VLAOPT'
    #else:
    #    telscope= 'good'
    
    outFile = cvReg.convolve(fileName,mainBeam,telescope)
    print outFile
    if outFile != fileBaseRg:
        rgFileNames.append(cvReg.regrid(fileBaseRg,outFile))
    else:
        rgFileNames.append(outFile)
    
    #ableFileNames.columns[0][i] = rgFileNames[i]

    #print tableFileNames.columns[0][i]
tableFileNames['FileName'] = tableFileNames['FileName'].astype(str)  
tableFileNames['FileName'] = rgFileNames
print tableFileNames['FileName']
#tableFileNames.add_column(rgFileNames, index=0)                                 
#tableFileNames.columns[0] =  rgFileNames
print tableFileNames.columns[0]
ascii.write(tableFileNames,tableNameOut,format='csv', overwrite=True)

print '''\t+---------+\n\t Convolution done\n\t+---------+'''


[0.00258542 0.00219688] [0.00466484 0.00241739]
ddd
(3600, 3600)
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Meer_1030MHz_pbcorr_cv.fits


[0.00188841 0.00159888] [0.00466484 0.00241739]
ddd
(3600, 3600)
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Meer_1440MHz_pbcorr_cv.fits
[0.00108611 0.00108611] [0.00466484 0.00241739]
ddd
(674, 664)
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_4860MHz_pbcorr_cv.fits
[0.00113056 0.00113056] [0.00466484 0.00241739]
ddd
(212, 202)
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_15GhzB_pbcorr_cv.fits
[0.00466484 0.00241739] [0.00466484 0.00241739]
ccc
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_101GHz.fits


/home/maccagni/programs/anaconda/lib/python2.7/site-packages/reproject/spherical_intersect/core.py:180: RuntimeWarning: divide by zero encountered in true_divide
  array_new /= weights


[0.00424951 0.00211456] [0.00466484 0.00241739]
ddd
(126, 108)
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_103GHz_cv.fits
[0.00460143 0.00234351] [0.00466484 0.00241739]
ddd
(126, 108)
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_113GHz_cv.fits
                                       FileName                                       
--------------------------------------------------------------------------------------
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Meer_1030MHz_pbcorr_cv_rg.fits
/home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Meer_1440MHz_pbcorr_cv_rg.fits
 /home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_4860MHz_pbcorr_cv_rg.fits
  /home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/VLA_15GhzB_pbcorr_cv_rg.fits
           /home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_101GHz_rg.fits
        /home/maccagni/Projects/MFS/FornaxA/contData/Core/inIms/Alma_103GHz_cv_rg.fits
        /home/maccagni/Pro